## GOAL
Create entanglement (Bell state) and observe simulator counts.


In [ ]:
# Import tools.
from qiskit import QuantumCircuit
from qiskit.primitives import Sampler
from qiskit.visualization import plot_histogram

# Build Bell circuit.
bell = QuantumCircuit(2, 2)
bell.h(0)
bell.cx(0, 1)
bell.measure([0, 1], [0, 1])

# Run on simulator.
sampler = Sampler()
res = sampler.run([bell], shots=1024).result()
counts = {k: int(v * 1024) for k, v in res.quasi_dists[0].items()}
print("Simulator counts:", counts)
plot_histogram(counts)


## EXERCISE
What results do you expect ideally? Why only two bitstrings?


## GOAL
Compare simulator vs real QPU for the Bell state.


In [ ]:
# Import runtime tools.
import os
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2, Session
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Get token and stop if missing.
token = os.getenv("QISKIT_IBM_TOKEN")
if not token:
    raise SystemExit("Token missing. Set QISKIT_IBM_TOKEN and restart the kernel.")

# Create the service.
service = QiskitRuntimeService(channel="ibm_quantum", token=token)

# Pick a real backend.
backend = service.least_busy(simulator=False, operational=True)
print("Using backend:", backend.name)

# Transpile for hardware.
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
bell_t = pm.run(bell)

# Run on hardware.
with Session(service=service, backend=backend) as session:
    sampler2 = SamplerV2(session=session)
    job = sampler2.run([bell_t], shots=1024)
    print("Job ID:", job.job_id())
    result_h = job.result()

# Convert to approximate counts.
quasi_h = result_h.quasi_dists[0]
counts_h = {k: int(v * 1024) for k, v in quasi_h.items()}
print("QPU counts (approx):", counts_h)
plot_histogram(counts_h)


## EXERCISE
Compute the percentage of (00 + 11) out of total counts.


## GOAL
Send 2 classical bits using 1 qubit (superdense coding).

Encoding table:
- 00 -> I
- 01 -> X
- 10 -> Z
- 11 -> XZ


In [ ]:
# Define superdense coding circuit.
def superdense(msg):
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    if msg == "01":
        qc.x(0)
    elif msg == "10":
        qc.z(0)
    elif msg == "11":
        qc.x(0)
        qc.z(0)
    qc.cx(0, 1)
    qc.h(0)
    qc.measure([0, 1], [0, 1])
    return qc

# Test all four messages.
for msg in ["00", "01", "10", "11"]:
    qc = superdense(msg)
    res = sampler.run([qc], shots=1024).result()
    counts = {k: int(v * 1024) for k, v in res.quasi_dists[0].items()}
    print("Message", msg, "->", counts)


## EXERCISE
Test all four messages and confirm decoding. Optional: run one message on QPU.
